# Compare results against larch

We use estimation data as prepared by larch and draw the Asim way and our way, then compare to larch probabilities.


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [10]:
import os
import argparse

import numpy as np
import pandas as pd

from activitysim.cli import run
from activitysim.core import inject

from fru_utils import mode_choice_for_trip

pd.set_option("max_columns", 500)

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_sf")
os.chdir(example_dir)

In [7]:
#!activitysim run -c configs -o output -d data_sf

In [8]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data_sf'])
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [68]:
larch_chooser_data = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "choosers_larch.csv"))
larch_probs = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "probabilities_larch.csv"), index_col=0)
# __caseids__ are tour ids, with trips in same order
assert (larch_probs.index.values ==  larch_chooser_data.tour_id).all()
larch_probs.index = larch_chooser_data.trip_id

In [50]:
larch_tours = pd.read_csv(os.path.join(root_dir, "test_estimation_sf", "output", "estimation_data_bundle",
                                       "trip_mode_choice", "trip_mode_choice_values_combined.csv"))
trips_merged_cols = ['person_id', 'household_id', 'primary_purpose', 'trip_num', 'outbound',
                     'trip_count', 'destination', 'origin', 'tour_id', 'purpose',
                     'destination_logsum', 'depart', 'hhsize', 'age', 'auto_ownership',
                     'number_of_participants', 'tour_category', 'parent_tour_id',
                     'tour_mode', 'duration', 'value_of_time', 'tour_type',
                     'free_parking_at_work', 'trip_period', "trip_id"]

larch_trips_merged = larch_tours[trips_merged_cols].set_index("trip_id")

In [87]:
def compare_mode_share_simulation(trip_id_to_check, num_samples, larch_trips_merged, larch_probs):
    t_, c_, ns_ = mode_choice_for_trip(choose_individual_max_utility=True, trip_id_to_check=trip_id_to_check,
                                       num_samples=num_samples, trips_merged=larch_trips_merged)
    sim_mode_shares = c_.trip_mode.value_counts() / c_.shape[0]

    t_asim, c_asim, _ = mode_choice_for_trip(choose_individual_max_utility=False, trip_id_to_check=trip_id_to_check,
                                             num_samples=num_samples, trips_merged=larch_trips_merged)
    asim_mode_shares = c_asim.trip_mode.value_counts() / c_asim.shape[0]

    larch_mode_shares = larch_probs.loc[larch_probs.index == trip_id_to_check].T
    larch_mode_shares.columns = ["larch"]

    comp = sim_mode_shares.to_frame("me")\
        .merge(asim_mode_shares.to_frame("asim"), left_index=True, right_index=True,how="outer")\
        .merge(larch_mode_shares,left_index=True, right_index=True, how="outer").fillna(0)

    return comp

In [ ]:
%%time

trip_id_to_check = 23695361
num_samples = 10000

comp = compare_mode_share_simulation(trip_id_to_check, num_samples, larch_trips_merged, larch_probs)
comp.loc[(comp != 0).any(axis=1), :].applymap(lambda x: f"{x:.3%}")

estimation bundle trip_mode_choice not in settings file estimation.yaml


trip_mode_choice tour_type 'work' (10000 trips)
